# Sasto deal web scraper

In [14]:
import requests
from bs4 import BeautifulSoup

In [216]:
URL = "https://www.sastodeal.com/sd-fast/food-essentials/dry-fruits.html?p=6&is_scroll=1"
response = requests.get(URL)

In [217]:
response.status_code

200

In [218]:
if response.status_code == 200:
    response_data = response.text
    print(response_data[:500])
else:
    print("Error")

 <!doctype html><html lang="en"><head > <meta charset="utf-8"/>
<meta name="title" content="Dry Fruits | Food Essentials | Daily Needs | Sastodeal"/>
<meta name="description" content="Shop various Dry Fruits from Food Essentials | Daily Needs products only at your local online shopping web-store in Nepal. Visit now!"/>
<meta name="keywords" content="Dry Fruits | Food Essentials | Daily Needs | Sastodeal | Online Shopping in Nepal | Buy Products in Nepal"/>
<meta name="robots" content="INDEX,FOLL


In [219]:
html_resp = BeautifulSoup(response.text, "html.parser")
html_resp.title.text

'Dry Fruits | Food Essentials | Daily Needs | Sastodeal'

In [220]:
# parsing all elements of class "product details product-item-details"

products = html_resp.find_all("div",{'class':"product details product-item-details"})
product_num = 1
len(products), products[product_num].find("strong",{"class":"product name product-item-name"})

(17,
 <strong class="product name product-item-name"><a class="product-item-link" href="https://www.sastodeal.com/wallnut-500gm-imported-gh-dfn-08-23.html">Wallnut 500Gm ( Imported)</a></strong>)

In [ ]:
# product name 
product_name = products[product_num].find("strong",{"class":"product name product-item-name"}).text
detail_link = products[product_num].find("strong",{"class":"product name product-item-name"}).find('a')['href']
{"product_name":product_name, "detail_link": detail_link}

In [ ]:
#product old price
price_box = products[product_num].find("div",{"class":"price-box price-final_price"})
try:
    newPrice = price_box.find("span",{"data-price-type":"finalPrice"})["data-price-amount"]
except TypeError:
    newPrice = None
try:
    oldPrice = price_box.find("span",{"data-price-type":"oldPrice"})["data-price-amount"]
except TypeError:
    oldPrice = None
    
{"old price " : oldPrice,"new price": newPrice}

# Detail page Scraping

In [234]:
# URL = detail_link
URL = "https://www.sastodeal.com/britannia-digestive-500gm-sd-gurjbis-002.html"
response = requests.get(URL)
if response.status_code == 200:
    response = BeautifulSoup(response.text,"html.parser")
    print(response.title.text)
# else:
    print("Error")

Britannia Digestive 500GM | Biscuit & Cookies | Sastodeal
Error


In [235]:
def text_parser(html_element):
    return html_element.text if html_element else None

In [239]:
#product info class = "product-info-main"
detail_info_raw = response.find("div",{"class":"row"})
data = {}

#vendor info
vendor_shipping_raw = detail_info_raw.find("div",{"class":"ratingsOut estimateRate"})
                 
vendor_raw = vendor_shipping_raw.find("a",{"id":"profileconnect","class":"shoptitle"})
data["vendor"] = vendor_raw.text
data["vendor link"] = vendor_raw["href"] 

shipping_from_raw = vendor_shipping_raw.find_all("p")[1]
data["shipping_from"] = shipping_from_raw.text[14:].strip() if shipping_from_raw.text.startswith("Shipping from:") else None
data['stock_availability'] = detail_info_raw.find("div",{"class":"product-info-price"}).\
                             find("span", {"class":"stockqty"}).find("span").text

data["detail"] = [raw_info.text for raw_info in detail_info_raw.find_all("div",{"class":"product attribute overview"})]
# data["raw detail"] = detail_info_raw.find("div",{"class":"product attribute overview"})

#description
try:
    data["description"] = detail_info_raw.find("div",{"class":"product attribute description"}).find("p").text
except:
    data["description"] = None
#additional attribute
additional_attribute_raw = detail_info_raw.find("div",{"id":"product-attribute-specs-table","class":"data table additional-attributes"})
data["brand"] = text_parser(additional_attribute_raw.find("span",{"data-th":"Brand"}))
data["type of product"] = text_parser(additional_attribute_raw.find("span",{"data-th":"Type Of Product"}))
data["nutritional facts"] = text_parser(additional_attribute_raw.find("span",{"data-th":"Nutritional Facts"}))
data["storing recommendation"] = text_parser(additional_attribute_raw.find("span",{"data-th":"Storing Recommendation"}))
data["country of origin"] = text_parser(additional_attribute_raw.find("span",{"data-th":"Country Of Origin"}))
data["warranty"] = text_parser(additional_attribute_raw.find("span",{"data-th":"Warranty"}))

data

{'vendor': 'Guruju Store',
 'vendor link': 'https://www.sastodeal.com/marketplace/seller/profile/shop/GurujuStore',
 'shipping_from': 'Kathmandu',
 'stock_availability': 'In Stock',
 'detail': [' Delivering inside Kathmandu Valley only\nBest for munching\nNutritious and healthy\nCrunchy and tasty\nLight and fulfilling\n\n'],
 'description': 'This digestive biscuit brings you five grains packed into a 500 gram pack of high nutrition.It contains coarse wheat flour and wheat bran which gives it a grainy texture along with oats and other ingredients like dried whey and palm oil, maltose syrup, salt, whole milk powder for that dose of calcium and protein, and raising agents and emulsifiers.',
 'brand': 'Generic',
 'type of product': 'Britannia Digestive',
 'nutritional facts': 'High in nutrients and flavours',
 'storing recommendation': 'Store at a room temperature of 70°F',
 'country of origin': None,
 'warranty': 'No Warranty '}

In [240]:
def remove_none(data):
    return {k:v for k,v in data.items() if v }

In [241]:
data_clean = remove_none(data)
data_clean

{'vendor': 'Guruju Store',
 'vendor link': 'https://www.sastodeal.com/marketplace/seller/profile/shop/GurujuStore',
 'shipping_from': 'Kathmandu',
 'stock_availability': 'In Stock',
 'detail': [' Delivering inside Kathmandu Valley only\nBest for munching\nNutritious and healthy\nCrunchy and tasty\nLight and fulfilling\n\n'],
 'description': 'This digestive biscuit brings you five grains packed into a 500 gram pack of high nutrition.It contains coarse wheat flour and wheat bran which gives it a grainy texture along with oats and other ingredients like dried whey and palm oil, maltose syrup, salt, whole milk powder for that dose of calcium and protein, and raising agents and emulsifiers.',
 'brand': 'Generic',
 'type of product': 'Britannia Digestive',
 'nutritional facts': 'High in nutrients and flavours',
 'storing recommendation': 'Store at a room temperature of 70°F',
 'warranty': 'No Warranty '}

In [ ]:
# field in sasto deal
sasto_deal_fields = [
     'vendor', 'vendor link', 'shipping_from',
     'stock_availability', 'detail', 'raw detail',
     'description', 'brand', 'type of product',
     'nutritional facts', 'storing recommendation',
     'warranty', "product_name", "detail_link",
     "old price ", "new price"
    ]
# daraz fields
daraz_fields = [
    "productUrl","name","nid", "image",
    "price","ratingScore","review", "location",
    "brandId","brandName","sellerId","sellerName"
]

# List of union of fields of both sasto deal and daraz 


union_fields = {
    "name": None,
    "description": None,
    "url": None,
    "additionalProperty":{
        "typeOfProduct":None,
        "nutritionalFacts":None,
        "storingRecommendation":None,
        "detail":None
        },
    "offers":{
        "price":None,
        "discount":None,
        "availability":None,
        "warranty":None
    }
    "brand":None,
    "seller":None,
    "image":None,
    "location":None,
    "review":None,
    "rating":None
    }